# Phase 1 — Modélisation du prix au m² à Lille (2022) pour les logements de 4 pièces

Dans cette première phase, vous allez construire un **modèle de prédiction du prix au m²** à partir de données issues des **ventes immobilières à Lille**, en vous concentrant sur les **logements de 4 pièces** vendus en **2022**.

Deux études séparées seront menées :

- L’une sur les **appartements**
- L’autre sur les **maisons**

---

### Objectifs

- Comprendre les relations entre certaines caractéristiques des biens (surface, nombre de lots, etc.) et leur prix au m²
- Comparer les dynamiques de prix entre appartements et maisons
- Mettre en œuvre un pipeline de modélisation supervisée
- Sélectionner les modèles les plus performants par type de bien
- Poser les bases d’un futur service d’estimation différencié par catégorie de logement

### Consignes

1. **Charger les données** depuis le fichier `data/lille_2022.csv`

In [ ]:
import pandas as pd
import numpy as np

data_lille = pd.read_csv("../data/lille_2022.csv")

# Afficher les 5 premières lignes
print("Aperçu des 5 premières lignes :")
print(data_lille.head())

# Afficher les informations sur le DataFrame
print("\nInformations sur le DataFrame :")
print(data_lille.info())

# Afficher les dimensions du DataFrame
print("\nDimensions du DataFrame (lignes, colonnes) :")
print(data_lille.shape)

Aperçu des 5 premières lignes :
   Identifiant de document  Reference document  1 Articles CGI  \
0                      NaN                 NaN             NaN   
1                      NaN                 NaN             NaN   
2                      NaN                 NaN             NaN   
3                      NaN                 NaN             NaN   
4                      NaN                 NaN             NaN   

   2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
0             NaN             NaN             NaN             NaN   
1             NaN             NaN             NaN             NaN   
2             NaN             NaN             NaN             NaN   
3             NaN             NaN             NaN             NaN   
4             NaN             NaN             NaN             NaN   

   No disposition Date mutation Nature mutation  ...  Nombre de lots  \
0               1    03/01/2022           Vente  ...               0   
1           


2. **Filtrer les biens de 4 pièces uniquement** : `Nombre pieces principales == 4`

In [ ]:
print("Valeurs uniques du nombre de pièces :")
print(data_lille["Nombre pieces principales"].unique())
print("\nNombre de biens par nombre de pièces :")
print(data_lille["Nombre pieces principales"].value_counts().sort_index())

data_4pieces = data_lille[data_lille["Nombre pieces principales"] == 4]

# Vérifier le résultat
print("\nNombre de biens de 4 pièces :", len(data_4pieces))
print("\nAperçu des biens de 4 pièces :")
print(data_4pieces.head())

Valeurs uniques du nombre de pièces :
[ 5.  3.  0.  2.  1.  4.  7.  6.  8. 10. 14.  9. 12. 11. 13.]

Nombre de biens par nombre de pièces :
Nombre pieces principales
0.0     5596
1.0     1471
2.0     1635
3.0     1273
4.0      789
5.0      379
6.0      139
7.0       42
8.0       11
9.0        9
10.0       4
11.0       2
12.0       1
13.0       1
14.0       2
Name: count, dtype: int64

Nombre de biens de 4 pièces : 789

Aperçu des biens de 4 pièces :
    Identifiant de document  Reference document  1 Articles CGI  \
18                      NaN                 NaN             NaN   
23                      NaN                 NaN             NaN   
26                      NaN                 NaN             NaN   
58                      NaN                 NaN             NaN   
62                      NaN                 NaN             NaN   

    2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
18             NaN             NaN             NaN             NaN   
23 


3. **Créer deux jeux de données distincts** :

    - Un jeu avec uniquement les **appartements**
    - Un jeu avec uniquement les **maisons**


In [14]:
print("Types de biens disponibles :")
print(data_4pieces["Type local"].unique())

data_appartements_4pieces = data_4pieces[data_4pieces["Type local"] == "Appartement"]

data_maisons_4pieces = data_4pieces[data_4pieces["Type local"] == "Maison"]

# Vérifier le résultat
print("\nNombre de appartements de 4 pièces :", len(data_appartements_4pieces))
print("\nAperçu des appartements de 4 pièces :")
print(data_appartements_4pieces.head())
print("\nNombre de maisons de 4 pièces :", len(data_maisons_4pieces))
print("\nAperçu des maisons de 4 pièces :")
print(data_maisons_4pieces.head())

Types de biens disponibles :
['Maison' 'Appartement']

Nombre de appartements de 4 pièces : 435

Aperçu des appartements de 4 pièces :
     Identifiant de document  Reference document  1 Articles CGI  \
23                       NaN                 NaN             NaN   
26                       NaN                 NaN             NaN   
58                       NaN                 NaN             NaN   
62                       NaN                 NaN             NaN   
113                      NaN                 NaN             NaN   

     2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
23              NaN             NaN             NaN             NaN   
26              NaN             NaN             NaN             NaN   
58              NaN             NaN             NaN             NaN   
62              NaN             NaN             NaN             NaN   
113             NaN             NaN             NaN             NaN   

     No disposition Date muta


4. Pour chaque jeu, ne conservez que les colonnes suivantes :
    - `Surface reelle bati`
    - `Nombre pieces principales`
    - `Type local`
    - `Surface terrain` (si disponible)
    - `Nombre de lots`
    - `Valeur fonciere` (pour calculer le `prix_m2`)

In [19]:
colonnes_à_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Type local',
    'Surface terrain',
    'Nombre de lots',
    'Valeur fonciere'
]

df_appartements_4pieces = data_appartements_4pieces[colonnes_à_garder].copy()
print(df_appartements_4pieces.columns)
print(df_appartements_4pieces.info())

df_maisons_4pieces = data_maisons_4pieces[colonnes_à_garder].copy()
print(df_maisons_4pieces.columns)
print(df_maisons_4pieces.info())

Index(['Surface reelle bati', 'Nombre pieces principales', 'Type local',
       'Surface terrain', 'Nombre de lots', 'Valeur fonciere'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 23 to 11351
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Surface reelle bati        435 non-null    float64
 1   Nombre pieces principales  435 non-null    float64
 2   Type local                 435 non-null    object 
 3   Surface terrain            49 non-null     float64
 4   Nombre de lots             435 non-null    int64  
 5   Valeur fonciere            435 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 23.8+ KB
None
Index(['Surface reelle bati', 'Nombre pieces principales', 'Type local',
       'Surface terrain', 'Nombre de lots', 'Valeur fonciere'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 354 entries, 18 to 

5. **Créer la variable cible** :
    
    ```python
    prix_m2 = Valeur fonciere / Surface reelle bati
    ```


In [21]:
# Ajouter la colonne prix_m2
df_appartements_4pieces['prix_m2'] = df_appartements_4pieces['Valeur fonciere'] / df_appartements_4pieces['Surface reelle bati']
df_maisons_4pieces['prix_m2'] = df_maisons_4pieces['Valeur fonciere'] / df_maisons_4pieces['Surface reelle bati']

# Vérifier les résultats
print("Statistiques prix/m² appartements :")
print(df_appartements_4pieces['prix_m2'].describe())
print("\nStatistiques prix/m² maisons :")
print(df_maisons_4pieces['prix_m2'].describe())

Statistiques prix/m² appartements :
count      435.000000
mean      4989.066032
std       7087.899307
min          0.011494
25%       2617.078231
50%       3285.714286
75%       4310.382231
max      52065.934066
Name: prix_m2, dtype: float64

Statistiques prix/m² maisons :
count     354.000000
mean     3003.594635
std      1116.900260
min       144.144144
25%      2291.390977
50%      2847.948718
75%      3481.995492
max      9192.073171
Name: prix_m2, dtype: float64


6. **Nettoyer les données** :
    - Supprimer les lignes avec valeurs manquantes sur les colonnes utilisées
    - Identifier et retirer les valeurs aberrantes (prix au m² trop faible ou trop élevé)


**7. Préparer les données pour l'entraînement**

- Variables explicatives : `X`
- Variable cible : `y = prix_m2`
- Division en jeu d'entraînement (80%) et test (20%) avec `train_test_split`

**8. Entraîner les modèles de base avec `scikit-learn`**

- `LinearRegression`
- `DecisionTreeRegressor`
- `RandomForestRegressor`

**9. Optimiser les modèles d’arbres avec `GridSearchCV`**

- Appliquer une recherche d’hyperparamètres sur les arbres pour améliorer les résultats

**10. Ajouter un modèle moderne : `XGBRegressor`**

🧑🏻‍💻 **Lien utile** : [https://www.ibm.com/fr-fr/think/topics/xgboost#:~:text=XGBoost (eXtreme Gradient Boosting) est,utilise la descente de gradient](https://www.ibm.com/fr-fr/think/topics/xgboost#:~:text=XGBoost%20(eXtreme%20Gradient%20Boosting)%20est,utilise%20la%20descente%20de%20gradient)

- Utilisé aujourd’hui dans de nombreux projets industriels
- Formation avec les mêmes données d'entraînement
- Prédiction et évaluation sur les données de test

**11. Évaluer les performances de tous les modèles**

- Utiliser la métrique **MSE** (`mean_squared_error`)
- Comparer les performances de tous les modèles testés
- Afficher un **tableau comparatif clair** pour :
    - les **appartements**
    - les **maisons**